In [2]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

http://decod.es/	v0.2.3
io loaded


# The Raster Family
The raster is a device in computer graphics that employs arrays of pixel values to represent form and image. It is a fundamentally different approach to describing form than the vector-based graphics we have discussed so far.

All the types in the Decod.es Raster family offer some variation on a mechanism for storing and manipulating data in a multi-dimensional array (a two-dimensional rectangular homogeneous matrix). Here we will focus our efforts on developing a family of bespoke types that follow this format, and that support routines unique to the two-dimensional matrix most commonly employed in visual design. Beyond illustrating the distinct issues in raster and vector implementations of geometry, each of these types enjoys some amount of utility in visual design applications.

***Image***, is ubiquitous in computer graphics.

The similarly-structured ***ValueField*** and ***BoolField*** types are more often appropriate choices for the sort of visual design routines presented in this text, and we may find such structures at work in applications that operate on abstract matrices of data, such as cellular automata and reaction-diffusion models. 

The remaining types presented, ***Grid*** and ***VecField***, foreground another distinction between the raster and vector models, and offer one approach to their synthesis.


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P22.jpg" style="width: 200px; display: inline;">


There is a distinction to be made between the pixel dimension of an image and the physical dimension assigned to it.

While the physical dimension may be rescaled without recomputing pixels, thereby manipulating the size of the image when printed but not altering the underlying information, the pixel dimension is directly bound to the underlying data. The relationship between these two dimensions is expressed as a ratio (such as pixels-per-inch) termed the ***resolution***. The concept of an arbitrary resolution being assigned to an image highlights the distinction between rasters, which are fundamentally dimensionless, and vectors, which describe defined positions in space.

In order to bring raster and vector together, it is necessary to establish a way to define a resolution. The abstract ***Grid*** type does just this by integrating a spatial ***Bounds*** into the ***Raster*** data structure. The ***VecField*** type demonstrates one application of such a spatialization of a Raster.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P30.jpg" style="width: 200px; display: inline;">


## Rasters

In Decod.es, ***Raster*** is the progenitor of all the classes presented in this section, and is an abstract class that provides mechanisms to store and access generic data in a raster grid format.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.C05.jpg" style="width: 200px; display: inline;">

The primary responsibility of the Raster is to manage access to the data structure `rstr._pixels` that will be used by all descendant types to store various kinds of objects.

This is stored in a private member `_pixels`, and accessed via the `get` and `set` methods.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P07.jpg" style="width: 200px; display: inline;">


In [1]:
"""
Raster Get and Set Methods
The translation from a given x,y coordinate of a pixel and its index in the 
_pixel collection may be obtained by a simple equation that accounts for the 
number of pixels per row of the Raster.
"""
def get(self,x,y):
    return self._pixels[y*self._dim[0]+x]

def set(self,x,y,value):
    self._pixels[y*self.px_width+x] = value

These methods require a description of the number of pixels in the
grid in x and y dimensions, which is stored as a Tuple of two Integers
in the private member `rstr._dim`.

In [ ]:
"""
Raster Pixel Dimension Properties
A number of properties are defined to ease access to the pixel dimension of a 
Raster.
"""
@property
def px_dim(self):
    return self._dim
    
@property
def px_width(self):
    return int(self._dim[0])

@property
def px_height(self):
    return int(self._dim[1])
    
@property
def px_count(self):
    return self.px_width*self.px_height

A number of useful but less central attributes are defined. 

Many of these are matters of convenience, such as the various ways of accessing the pixel dimension found in the properties above and below.

In [ ]:
"""
Raster Pixel Addresses
Many processes require iteration over every available pixel in a Raster. Here, 
all the valid pixel addresses are returned as a single collection.
"""
@property
def addresses(self):
    return itertools.product(range(self.px_width),range(self.px_height))

Two other methods are worth describing in more detail.

The `rstr.populate(val)` method fills each pixel address of an existing Raster with a given value. 

The `rstr.neighbors_of(x,y)` method is responsible for returning those objects that occupy addresses adjacent to a given address.

In [ ]:
"""
Raster Population
Here, every pixel in a Raster is populated with a given value.
"""
def populate(self,val):
    self._pixels = [val]*self.px_count

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P31.jpg" style="width: 200px; display: inline;">


The Raster initialization requires only one named argument, `pixel_dim`, with the remainder of the variables handled as packed keyword arguments. 

The given `pixel_dim` may be an Interval, a Tuple of two numbers, or nothing at all. The distinction between these cases is primarily handled with a `try-except` structure. 

Next, the optional members associated with directing the production of pixel neighborhoods are defined. 

Finally, an empty collection of `rstr._pixels` is initialized, but not populated with values. 

***Since the type of object contained will vary, the population of values is left to the derived classes.***

In [ ]:
"""
Raster Initialization
The initialization of a Raster proceeds by first setting the pixel dimension 
and accounting for any given keyword arguments before initializing the _pixels 
collection. Note that this collection is defined here, but contains no objects. 
Populating the pixels of a Raster is left to descendant types.
"""
class Raster(object):
    def __init__(self,pixel_dim=(20,20),**kwargs):
        self._dim = pixel_dim
        self.include_corners = False
        if "include_corners" in kwargs: 
            self.include_corners = kwargs["include_corners"]
        self.wrap = False
        if "wrap" in kwargs: 
            self.wrap = kwargs["wrap"]
        
        # the _pixels collection is initialized but not populated
        self._pixels = []

### Raster Descendants

The Raster class is abstract, and not meant to be instantiated.

Among the simplest of the descendants of Raster are the ***BoolField***, ***ValueField***, and ***Image*** types, which are raster data structures for storing Booleans, numeric values, and Colors, respectively.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P24.jpg" style="width: 200px; display: inline;">


Since the Raster class was left intentionally incomplete, and did not populate the `rstr._pixels` member with objects, a minimal implementation of a class that inherits Raster consists simply of a constructor that extends Raster by calling its parent’s initialization method, and then populates this collection with the appropriate type of object.

In [ ]:
"""
Initialization of Raster Descendants
The direct descendants of Raster present very similar initialization routines. 
Each calls the initialization of its parent before populating the _pixels 
collection with an appropriate object type. Optionally, as we seen in BoolField
and ValueField, the default values of keyword arguments may be adjusted.
"""
class BoolField(Raster):
    def __init__(self, pixel_dim=None, initial_value=False, **kwargs):
        if "wrap" not in kwargs: kwargs["wrap"] = True
        super(BoolField,self).__init__(pixel_dim,**kwargs)
        self.populate(initial_value)
        
class ValueField(Raster):
    def __init__(self, pixel_dim=None, initial_value=0.0, **kwargs):
        if "wrap" not in kwargs: kwargs["wrap"] = True
        super(ValueField,self).__init__(pixel_dim,**kwargs)
        self.populate(initial_value)

class Image(Raster):
    def __init__(self,pixel_dim,initial_color = Color(),**kwargs):
        super(Image,self).__init__(pixel_dim,**kwargs)
        self.populate(initial_color)

Each sub-class is now free to define its own set of methods or members as appropriate to the type it stores. 

For example, ***ValueField*** defines routines for determining the minimum and maximum values stored, while ***Image*** defines a method for saving the contained data as a file

In [ ]:
"""
Unique Methods of a ValueField
Some descendants of Raster require methods that are only appropriate to their 
contained type. Here we see two methods that are unique to the ValueField type.
"""
@property
def max_value(self):
    return max(self._pixels)

@property
def min_value(self):
    return min(self._pixels)

## Grids

Raster images are fundamentally dimensionless. 

Other than the number of pixels contained by the two dimensions of a raster matrix, any assigned spatial dimension is arbitrary, and may be altered without affecting the data underlying the raster. 

For this reason, the Raster, BoolField, ValueField, and Image classes may be seen as non-geometric and do not interact with the space of most of the geometry found in this text. To bring a raster object into space, we must assign it a spatial dimension. 

This is exactly the role of the Decod.es Grid, which is a descendant of the Raster type, but one that integrates a dimensional object called a Bounds into its data structure.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P25.jpg" style="width: 200px; display: inline;">


### Bounds Objects in Decod.es

Decod.es offers a utility class which operates as a higher dimensional version of the Interval, the ***Bounds***, which manifests as the coupling of two or three Intervals in order to provide a two-dimensional rectangular or three-dimensional cubic structure.

Each object of type Bounds stores two or three primary Interval members - `bnds.ival_x`, `bnds.ival_y`, and optionally `bnds.ival_z` - and is constructed using as many intervals as the dimension of the space. 

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D88 Bounds Large.jpg" style="width: 800px; display: inline;">


Functionality is provided to construct Bounds objects in a variety of ways, such as `Bounds.unit_square()` and `Bounds.unit_cube()`, or by passing a collection of encompassed Points.

In [3]:
"""
Construction of a Decod.es Bounds
A 2d Bounds is constructed by two intervals, a 3d bounds by three. 
Alternatively, we may construct a Bounds by centerpoint and dimension.
"""

bnds_2d = Bounds(ival_x=Interval(-2,2), ival_y=Interval(-6,1))
bnds_3d = Bounds(ival_x=Interval(), ival_y=Interval(2,1), ival_z=Interval())
bnds_by_cpt = Bounds(center=Point(), dim_x=3.0, dim_y=1.5)

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P32.jpg" style="width: 200px; display: inline;">


The arguments used in these constructions are also made available to any Bounds objects as attributes.

In [4]:
"""
Bounds Attributes
"""
print bnds_2d.cpt
print bnds_3d.dim_x
print bnds_by_cpt.ival_x

pt[0.0,-2.5,0]
1.0
ival[-1.5,1.5]


In [ ]:
"""
Bounds Encompass
"""
pts = [Point(-1,2), Point(-3,0), Point(2,2), Point(0,4)]
bnds = Bounds.encompass(pts)

### Grid Objects in Decod.es

Although ***Grid*** is a Raster descendant, unlike the concrete classes of BoolField, ValueField, and Image, it does not assign a particular object type to be stored, but merely provides an abstract framework from which other types may be derived. 

It is the ***abstract descendant of an abstract class***. 

We will use the ***VecField*** type, which stores Decod.es Vecs in a Grid structure, to demonstrate the utility of such a construct.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.C06.jpg" style="width: 200px; display: inline;">

The Grid type is responsible for managing access to a spatial Bounds `grd._bnds`, and for providing methods that combine queries on this space with the data contained within the inherited raster structure. In this way, each pixel in the raster data is associated with a subdivision of the spatial Bounds that we’ll term a grid cell. Many of the methods offered by a Grid concern transformations from spatial coordinates relative to the Bounds to pixel coordinates of data in the Raster, or vice-versa.

Two members are calculated at initialization, and stored as private members: 

* A set of base points `_base_pts` at the center of each Grid cell 
* A set of pre-divided Intervals, `_ivals_x` and `_ivals_y`, that describe divisions in the x- and y-dimensions.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P33.jpg" style="width: 200px; display: inline;">


The initialization of a Grid closely resembles that of its predecessors.

In [5]:
"""
Grid Initialization
This abstract class for storing information in a spatialized raster grid format
is constructed first by calling the Raster initialization, and then establishing 
a two-dimensional spatial bounds _bnds. Finally, a number of values concerning 
the division of this spatial bounds are pre-calculated.
"""
class Grid(Raster):
    def __init__(self,pixel_dim=None,bnds=None,**kwargs):
        super(Grid,self).__init__(pixel_dim,**kwargs)
        if bnds is None: self._bnds = Bounds.unit_square()
        # enforces a two-dimensional Bounds
        else: self._bnds = Bounds(ival_x=bnds.ival_x,ival_y=bnds.ival_y ) 
        self._recalculate_base_pts()

Calculating values in advance eases the burden on subsequent method calls, but at the cost of re-calculating these values any time `grd._bnds` is altered.

In [ ]:
"""
Base Point Calculation
A Grid pre-calculates values that describe the center Point of each grid cell, 
as well as divisions of the overall x and y Intervals. This pre-calculation 
eases the burden on subsequent method calls, but must be re-calculated any time 
the stored _bnds is altered.
"""
def _recalculate_base_pts(self):
    self._base_pts = []
    self._ivals_x = self.bnds.ival_x//self.px_width
    self._ivals_y = self.bnds.ival_y//self.px_height
    for ival_y in self._ivals_y:
        for ival_x in self._ivals_x:
            self._base_pts.append(Point(ival_x.mid, ival_y.mid))

The remainder of the methods offered by Grid concern the translation between spatial and pixel coordinates. 

The `grd.get_cpt()` method, for example, allows us to move from the raster to the spatial. Given a pixel address, this routine produces the center Point of the spatial grid cell associated with it.

In [ ]:
"""
Gridcell Center Point
A method that returns the center point of the cell associated with the given 
address.
"""
def get_cpt(self,x,y):
    return self._base_pts[y*self.px_width+x]

The `grd.address_near()` method is effectively the inverse of this process: given a spatial position, it calculates the pixel address of the containing grid cell.

In [ ]:
"""
Grid Address Near
A method that returns the address of the grid cells nearest the given location.
This method may be passed either a point or an x,y coordinate.
"""    
def address_near(self,a,b=None):
    pt = Point(a,b)
    # find index of interval containing x-coordinate
    if pt.x <= self.bnds.ival_x.a : idx_x = 0
    elif pt.x >= self.bnds.ival_x.b : idx_x = self.px_width - 1
    else: idx_x = [pt.x in ival for ival in self._ivals_x].index(True)
    # find index of interval containing y-coordinate
    if pt.y <= self.bnds.ival_y.a : idx_y = 0
    elif pt.y >= self.bnds.ival_y.b : idx_y = self.px_height - 1
    else: idx_y = [pt.y in ival for ival in self._ivals_y].index(True)
    # return address tuple
    return idx_x, idx_y

The related `grd.addresses_near()` method operates in a similar manner, returning the addresses of all the grid cells that are near to a given spatial position

In [ ]:
"""
Grid Addresses Near
Here, a collection is returned containing addresses of grid cells near to the 
given location. This method may be passed either a point or an x,y coordinate.
"""         
def addresses_near(self,a,b=None):
    pt = Point(a,b)
    add = self.address_near(pt)
    dx = 1 if pt.x > self._ivals_x[add[0]].mid else -1
    dy = 1 if pt.y > self._ivals_y[add[1]].mid else -1
    adds = [add,(add[0]+dx,add[1]),(add[0]+dx,add[1]+dy),(add[0],add[1]+dy)]
    adds = filter(lambda add: add[0]>=0 and add[0]<self.px_width, adds)
    adds = filter(lambda add: add[1]>=0 and add[1]<self.px_height, adds)
    return sorted(adds)

Some methods require a round-trip from spatial to raster and back again, a process greatly simplified by the methods defined above.

Given a spatial coordinate, the `grd.cpts_near()` method first finds the pixel addresses of nearby grid cells, and then returns the center Points of each.

In [ ]:
"""
Grid Center Points Near
The center points of cells near the given location are returned by this method. 
Note the use of the splat operator to pack the pixel address tuple so that they
are passed as separate arugments to the get_cpt method.
"""       
def cpts_near(self,a,b=None):
    return [self.get_cpt(*add) for add in self.addresses_near(a,b)]

Discrete vector fields are natural structures to inherit from Grid. Such is the role of the VecField class. The `vecfld.__init__()` method is unremarkable, and faithfully follows the pattern established by the concrete classes above. 

Many of the other methods of VecField are wrappers for the analogous methods of Grid or modest modifications thereof. For example, the `vecfld.vec_near()` and `vecfld.vecs_near()` methods are straightforward wrappers for the related Grid methods, each tacking on a call to the raster structure such that they return Vec objects rather than pixel addresses.

In [ ]:
"""
Vec and Vecs Near
Two methods that return closest vector or the closest set of vectors to the 
given location. Again, the splat operator is employed for argument packing.
"""
def vec_near(self,a,b=None):
    return self.get(*self.address_near(a,b))

def vecs_near(self,a,b=None):
    return [self.get(*add) for add in self.addresses_near(a,b)]

The more elaborate method `vecfld.avg_vec_near()` builds on this functionality by calculating the average of those Vecs near to a given spatial position, and weighting them by their proximity to this position.

In [ ]:
"""
Average Vec Near
Returns an average vector from the near vectors around the given location. 
"""
def avg_vec_near(self,sample_pt):
    vecs = self.vecs_near(sample_pt)
    dists = [1.0/sample_pt.distance2(pt) for pt in self.cpts_near(sample_pt)]
    tot = sum(dists)
    weights = [dist/tot for dist in dists]
    
    vec = Vec()
    for v,w in zip(vec,weights): vec += v*w
    return vec
    

## Multiple Inheritance

The classes discussed thus far have followed a rational pattern. The abstract classes, Raster and Grid, have defined patterns of behavior that we expect will benefit other classes. This abstraction made things easy for the concrete classes (BoolField, ValueField, and VecGrid) which inherited much of their important functionality from their progenitors, needing only implement those behaviors unique to the types of objects that they store.

This approach has produced a rational chain of inheritance and a tidy family of types. 

As is often the case, ***disruption of a design pattern is only as far away as the nearest broken assumption***.

We have identified two categories of behavior in the design of these classes: those that encapsulate data structures (the matrix-like Raster and the spatialized matrix of the Grid) and those that extend a data structure to accommodate the needs of a stored type (ValueFields do number-like things on Rasters, while VecFields do Vec-like things on Grids).

***What would occur if we found we required a type that mixed these relationships in a way not yet anticipated?***

For example, what if we required a ValueGrid?


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P26.jpg" style="width: 200px; display: inline;">


Helpful in this particular case is a type of feature offered by many programming languages to allow for a form of multiple inheritance, which allows behavior to be inherited from more than one direct ancestor. 

Python supports this feature through an extension of the inheritance syntax with which we are already familiar.

    class SomeType( PrimaryParent, SecondaryParent, ... ):

We can see the potential complications that arise from this structure. The hierarchy implied by defining a `PrimaryParent` and `SecondaryParent` suggests that a mechanism is in place to untangle any conflicting inherited attributes.

Using multiple inheritance, defining the hybrid ValueGrid type is shockingly simple. By combining the inherited attributes of a Grid and a ValueField, we can manifest a working type in just one line of code.

This new type inherits the spatialized Raster structure of a Grid and the number-handling methods of a ValueField.

In [ ]:
"""
The Multiple Inheritance of a ValueGrid
In one simple statement we define the ValueGrid type, which inherits the 
spatialized raster structure of a Grid and the methods related to storing 
numeric values of a ValueField
"""
class ValueGrid(Grid, ValueField):

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P10.jpg" style="width: 200px; display: inline;">


While this version of a ValueGrid is useful on its own, it can be extended a bit by defining a method. Here, we define a new concept related to grid-like data structures: a ***quartet*** is a Tuple of values and Points that represent the information of four related ValueGrid cells.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P11.jpg" style="width: 200px; display: inline;">


In [ ]:
"""
Point-Value Quartet
A quartet represents the center points and values of four ValueGrid cells that 
meet at a corner. Since four cells are required, the effective pixel_dim of 
quartets is one fewer in each dimension than that of the ValueField which they 
describe.
"""
def quartet(self,x,y):
    idxs = [(x,y),(x+1,y),(x+1,y+1),(x,y+1)]
    # return a Tuple of (values, pts)
    return [self.get(*xy) for xy in idxs], [self.get_cpt(*xy) for xy in idxs]